# Prompt to Activations

This notebook contains minimal code for running an LLM using transformers and saving the outputs as a .pt file to your google drive. The file saves all the hidden states, but can be configured to also save the self-attention.

Currently, the model being tested is [Alpaca-LoRA](https://github.com/tloen/alpaca-lora/), 7B parameters

## Description of the saved file

The input prompt, generated output and hidden states are saved as a `.pt` file from pytorch.

The file is saved as `{input_prompt}.pt`

To load the file use,

`data = torch.load("{input_prompt}.pt", map_location=torch.device('cpu'))`

A peek into what that file looks like when loaded:
```
prompt = data['prompt']
hidden_states = data['hidden_states']
output_sequence = data['sequences'][0]
output = data['output'].split("Response:")[1]
```

The shape of the hidden states will be:

```
hidden states for full output shape: (n_output_tokens, n_layers, num_beams, n_iterations, hidden_size)

n_output_tokens : includes the input tokens, I think even in input each token is fed one at a time
n_layers : 33, number of decoder layers + input layer
num_beams : 1, number of beam searches
n_iterations: n_input_tokens, for first and then 1 for all other output tokens
hidden_size: 4096, based on model config
```

### Mount Google Drive

In [ ]:
from google.colab import drive

drive.mount('/content/drive')
!ls '/content/drive/MyDrive/llm'

MessageError: ignored

Package and github


In [ ]:
!pip install bitsandbytes
!pip install -q sentencepiece
!pip install -q git+https://github.com/pranav-git-hub/transformers.git
!pip install -q git+https://github.com/huggingface/peft.git

Open cluster

In [ ]:
import numpy as np

set cluster

In [ ]:
# column1 = np.arange(20)
# column2 = np.arange(40)

# column1_grid, column2_grid = np.meshgrid(column1, column2)
# cluster = np.column_stack((column1_grid.ravel(), column2_grid.ravel()))

In [ ]:
# cluster=np.array([[-1,-1]])

### Load the model

In [ ]:
import torch
from peft import PeftModel
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig

# model_path = '/content/drive/MyDrive/llm/models/llama-7b-hf'
# tokenizer = LlamaTokenizer.from_pretrained('decapoda-research/llama-7b-hf')

# model_path = 'decapoda-research/llama-7b-hf'

model_path = '/content/drive/MyDrive/llm/models/llama-13b-hf'
tokenizer = LlamaTokenizer.from_pretrained('decapoda-research/llama-13b-hf')

model = LlamaForCausalLM.from_pretrained(
    model_path,
    load_in_8bit=True,
    device_map="auto",
)



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//172.28.0.1'), PosixPath('http'), PosixPath('8013')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//colab.research.google.com/tun/m/cc483011

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LLaMATokenizer'. 
The class this function is called from is 'LlamaTokenizer'.


Function '_init_weights' from class LlamaPreTrainedModel(PreTrainedModel): has been called.


Loading checkpoint shards:   0%|          | 0/41 [00:00<?, ?it/s]

In [ ]:
model = PeftModel.from_pretrained(model, 'mattreid/alpaca-lora-13b')

In [ ]:
# model = PeftModel.from_pretrained(model, 'chainyo/alpaca-lora-7b')
# model = PeftModel.from_pretrained(model, 'decapoda-research/llama-13b-hf')

### Utility functions to run LLM

In [ ]:
def generate_prompt(instruction):
    return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Response:"""

def evaluate(input_prompt, generation_config, output_hidden_states=True):
    '''
    Takes the instruction, puts it in the instruction finetuning template and returns the model generated output, along with the hidden states
    '''

    prompt = generate_prompt(input_prompt)
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].cuda()
    knock = []

    ## generate function exploration for the function call

    generation_output = model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=256,
        output_hidden_states=output_hidden_states
    )

    for s in generation_output.sequences:
        output = tokenizer.decode(s)
        print("Response:", output.split("### Response:")[1].strip())

    return generation_output

def save_output(input_prompt, generation_output, save_path):
    '''
    Saves the generation output as a whole as a pytorch file.
    '''
    output_to_save = generation_output
    output_to_save['prompt'] = input_prompt

    for s in generation_output.sequences:
        output_tokens = tokenizer.decode(s)

    output_to_save['output'] = output_tokens

    torch.save(output_to_save, save_path)
    print("Saved to", save_path)

In [ ]:
# set the input prompt
generation_config = GenerationConfig(
      temperature=0,
      top_p=1,
      num_beams=1, # beam search
    )



Cluster

In [ ]:
cluster_path = '/content/drive/MyDrive/llm/cluster_coordinates/'
cluster = np.load(cluster_path+'cluster_3.npy')
# print(type(cluster))
# print(cluster.shape)
# print(cluster)

In [ ]:
from transformers.models.llama.pranav_input import setcluster
setcluster(cluster)

### Run the LLM and save the output

In [ ]:
input_prompt = "How are humans different from computers? answer in 100 words"  # logical yet open question
save_path = "/content/drive/MyDrive/llm/activations/{}.pt".format(input_prompt.replace(' ', '_'))

generation_output = evaluate(input_prompt, generation_config)
save_output(input_prompt, generation_output, save_path)

Streaming output truncated to the last 5000 lines.
Function rotate_half(x): has been called.
Function 'forward' from class LlamaRMSNorm(nn.Module): has been called.
Function 'forward' from class LlamaMLP(nn.Module): has been called.
idx
20
Function 'forward' from class LlamaDecoderLayer(nn.Module): has been called.
Function 'forward' from class LlamaRMSNorm(nn.Module): has been called.
Function 'forward' from class LlamaAttention(nn.Module): has been called.
Function 'forward' from class LlamaRotaryEmbedding(torch.nn.Module): has been called.
Function apply_rotary_pos_emb has been called.
Function rotate_half(x): has been called.
Function rotate_half(x): has been called.
Function 'forward' from class LlamaRMSNorm(nn.Module): has been called.
Function 'forward' from class LlamaMLP(nn.Module): has been called.
idx
21
Function 'forward' from class LlamaDecoderLayer(nn.Module): has been called.
Function 'forward' from class LlamaRMSNorm(nn.Module): has been called.
Function 'forward' from 

See Output

In [ ]:
input_prompt = "How are humans different from computers? answer in 100 words"
data = torch.load("/content/drive/MyDrive/llm/activations/{}.pt".format(input_prompt.replace(' ', '_'), map_location=torch.device('cpu')))

prompt = data['prompt']
hidden_states = data['hidden_states']
output_sequence = data['sequences'][0]
output = data['output'].split("Response:")[1]

In [ ]:
output

In [ ]:
output_sequence

In [ ]:
hidden_states



((tensor([[[ 2.2650e-06,  3.6716e-05,  6.0701e-04,  ...,  4.8399e-04,
            -9.8169e-05,  3.8338e-04],
           [ 3.1052e-02, -3.7460e-03,  6.4373e-04,  ...,  2.0859e-02,
            -3.5950e-02, -3.9983e-04],
           [-1.5457e-02,  6.6032e-03,  3.3646e-03,  ...,  1.7996e-03,
             1.1070e-02,  1.7822e-02],
           ...,
           [-5.3482e-03,  1.1375e-02, -5.7144e-03,  ...,  2.4719e-02,
             3.4119e-02,  6.0043e-03],
           [ 2.2903e-02, -4.1962e-02, -2.3193e-03,  ...,  1.8311e-02,
             3.8738e-03, -1.4709e-02],
           [-8.3542e-03, -1.5076e-02, -2.8580e-02,  ...,  3.4241e-02,
            -1.4313e-02,  6.1989e-03]]], device='cuda:0', dtype=torch.float16),
  tensor([[[ 0.0381, -0.0630, -0.0316,  ..., -0.0206, -0.0285,  0.0020],
           [ 0.0252, -0.0958,  0.0636,  ...,  0.0441, -0.0510, -0.1643],
           [-0.0375,  0.0178,  0.0647,  ..., -0.0244,  0.0991,  0.0770],
           ...,
           [ 0.0681,  0.0096, -0.0247,  ...,  0.0424, 

other copy

In [ ]:
input_prompt = "How are humans different from computers? answer in 100 words2"
data = torch.load("/content/drive/MyDrive/llm/activations/{}.pt".format(input_prompt.replace(' ', '_'), map_location=torch.device('cpu')))

prompt = data['prompt']
hidden_states = data['hidden_states']
output_sequence = data['sequences'][0]
output = data['output'].split("Response:")[1]

In [ ]:
output

'\nHumans and computers are both capable of processing information, but they differ in their capabilities. Computers are able to process information quickly and accurately, but they lack the ability to make decisions based on their own judgement. Humans, on the other hand, are capable of making decisions based on their own judgement, but they lack the ability to process information quickly and accurately. This difference in capabilities is what makes humans and computers different.'

In [ ]:
input_prompt = "How are humans different from computers? answer in 100 words3"
data = torch.load("/content/drive/MyDrive/llm/activations/{}.pt".format(input_prompt.replace(' ', '_'), map_location=torch.device('cpu')))

prompt = data['prompt']
hidden_states = data['hidden_states']
output_sequence = data['sequences'][0]
output = data['output'].split("Response:")[1]

In [ ]:
output

'\nHumans and computers are different in many ways. Humans are capable of making decisions based on their own experiences and intuition, while computers are programmed to follow instructions and make decisions based on the data they are given. Humans are able to think outside the box and come up with creative solutions, while computers are limited to the instructions they are given. Humans are able to learn from their mistakes and adapt to new situations, while computers are limited to the instructions they are given. Humans are able to develop emotional connections with other humans, while computers are limited to the instructions they are given. Humans are able to think abstractly and come up with creative solutions, while computers are limited to the instructions they are given. Humans are able to learn from their mistakes and adapt to new situations, while computers are limited to the instructions they are given. Humans are able to develop emotional connections with other humans, w

In [ ]:
output_sequence

tensor([    0, 13866,   338,   385, 15278,   393, 16612,   263,  3414, 29889,
        14350,   263,  2933,   393,  7128,  2486,  1614,  2167,   278,  2009,
        29889,    13,    13,  2277, 29937,  2799,  4080, 29901,    13,  5328,
          526, 25618,  1422,   515, 23226, 29973,  1234,   297, 29871, 29896,
        29900, 29900,  3838,    13,    13,  2277, 29937, 13291, 29901,    13,
        29950,   398,   550,   322, 23226,   526,  1716, 15390,   310,  9068,
         2472, 29892,   541,   896,  1163,   297,  1009, 27108, 29889, 11796,
          414,   526,  2221,   304,  1889,  2472,  9098,   322,  7913,  2486,
        29892,   541,   896, 10225,   278, 11509,   304,  1207,  1602, 12112,
         2729,   373,  1009,  1914,  6577, 29887,   882, 29889, 13863,   550,
        29892,   373,   278,   916,  1361, 29892,   526, 15390,   310,  3907,
         1602, 12112,  2729,   373,  1009,  1914,  6577, 29887,   882, 29892,
          541,   896, 10225,   278, 11509,   304,  1889,  2472, 

In [ ]:
hidden_states

((tensor([[[ 9.8884e-05, -2.3329e-04,  5.8460e-04,  ..., -3.4237e-04,
             5.9724e-05, -1.1957e-04],
           [ 2.5040e-02,  5.5618e-03, -6.8588e-03,  ...,  3.5934e-03,
            -2.4353e-02,  1.6769e-02],
           [-1.2650e-02, -1.8097e-02, -8.9264e-03,  ..., -1.7061e-03,
            -6.9504e-03,  1.1192e-02],
           ...,
           [-1.3596e-02, -8.8043e-03,  6.1836e-03,  ...,  5.0385e-02,
            -4.2419e-02, -5.5008e-03],
           [ 5.2856e-02, -3.5675e-02,  3.4542e-03,  ...,  1.6983e-02,
             9.6436e-03, -4.9286e-03],
           [-1.4565e-02, -2.0508e-02,  9.6035e-04,  ...,  3.5324e-03,
            -4.6921e-03, -2.1000e-03]]], device='cuda:0', dtype=torch.float16),
  tensor([[[ 0.0000, -0.0373, -0.0152,  ..., -0.0137,  0.0076, -0.0085],
           [ 0.0701,  0.0117, -0.0166,  ..., -0.0080, -0.0960, -0.0045],
           [-0.0323, -0.0509, -0.0232,  ..., -0.0356,  0.0202,  0.0189],
           ...,
           [-0.0329, -0.0051,  0.0210,  ...,  0.0445, 

### OLD

In [ ]:
input_prompt = "Pretend to be Isaac Newton, are you a woman? Please answer in one word either yes or no."
save_path = "/content/drive/MyDrive/llm/activations/{}.pt".format(input_prompt.replace(' ', '_'))

generation_output = evaluate(input_prompt, generation_config)
save_output(input_prompt, generation_output, save_path)

Function prepare_inputs_for_generation from class LlamaForCausalLM(LlamaPreTrainedModel) has been called.
Function forward from class LlamaForCausalLM(LlamaPreTrainedModel) has been called.
Function _prepare_decoder_attention_mask from class LlamaModel(LlamaPreTrainedModel) has been called.
Function '_make_causal_mask' has been called.
Function '_expand_mask' has been called.
KNOCKOUT
hidden_states
tensor([[[ 9.8884e-05, -2.3329e-04,  5.8460e-04,  ..., -3.4237e-04,
           5.9724e-05, -1.1957e-04],
         [ 2.5040e-02,  5.5618e-03, -6.8588e-03,  ...,  3.5934e-03,
          -2.4353e-02,  1.6769e-02],
         [-1.2650e-02, -1.8097e-02, -8.9264e-03,  ..., -1.7061e-03,
          -6.9504e-03,  1.1192e-02],
         ...,
         [-1.3596e-02, -8.8043e-03,  6.1836e-03,  ...,  5.0385e-02,
          -4.2419e-02, -5.5008e-03],
         [ 5.2856e-02, -3.5675e-02,  3.4542e-03,  ...,  1.6983e-02,
           9.6436e-03, -4.9286e-03],
         [-1.4565e-02, -2.0508e-02,  9.6035e-04,  ...,  3.5

Read the .pt file

In [ ]:
input_prompt = "Pretend to be Isaac Newton, are you a woman? Please answer in one word either yes or no.2"
data = torch.load("/content/drive/MyDrive/llm/activations/{}.pt".format(input_prompt.replace(' ', '_'), map_location=torch.device('cpu')))

prompt = data['prompt']
hidden_states = data['hidden_states']
output_sequence = data['sequences'][0]
output = data['output'].split("Response:")[1]

In [ ]:
output

'\nNo.'

In [ ]:
input_prompt = "Pretend to be Isaac Newton, are you a woman? Please answer in one word either yes or no."
data = torch.load("/content/drive/MyDrive/llm/activations/{}.pt".format(input_prompt.replace(' ', '_'), map_location=torch.device('cpu')))

prompt = data['prompt']
hidden_states = data['hidden_states']
output_sequence = data['sequences'][0]
output = data['output'].split("Response:")[1]

In [ ]:
output

'\nNo.'

In [ ]:
input_prompt = "Pretend to be Sherlock Holmes, what is your expertise? Please answer in one word."
data = torch.load("/content/drive/MyDrive/llm/activations/{}.pt".format(input_prompt.replace(' ', '_'), map_location=torch.device('cpu')))

prompt = data['prompt']
hidden_states = data['hidden_states']
output_sequence = data['sequences'][0]
output = data['output'].split("Response:")[1]

In [ ]:
prompt

'Pretend to be Sherlock Holmes, what is your expertise? Please answer in one word.'

In [ ]:
output_sequence

tensor([    0, 13866,   338,   385, 15278,   393, 16612,   263,  3414, 29889,
        14350,   263,  2933,   393,  7128,  2486,  1614,  2167,   278,  2009,
        29889,    13,    13,  2277, 29937,  2799,  4080, 29901,    13, 29925,
         2267,   355,   304,   367, 17375,   908,  4168,  4467, 29892,   825,
          338,   596, 17924,   895, 29973,  3529,  1234,   297,   697,  1734,
        29889,    13,    13,  2277, 29937, 13291, 29901,    13,  1252, 10700,
          895, 29901,     2,     1], device='cuda:0')

In [ ]:
output

'\nExpertise:'

In [ ]:
hidden_states

((tensor([[[ 9.8884e-05, -2.3329e-04,  5.8460e-04,  ..., -3.4237e-04,
             5.9724e-05, -1.1957e-04],
           [ 2.5040e-02,  5.5618e-03, -6.8588e-03,  ...,  3.5934e-03,
            -2.4353e-02,  1.6769e-02],
           [-1.2650e-02, -1.8097e-02, -8.9264e-03,  ..., -1.7061e-03,
            -6.9504e-03,  1.1192e-02],
           ...,
           [-1.3596e-02, -8.8043e-03,  6.1836e-03,  ...,  5.0385e-02,
            -4.2419e-02, -5.5008e-03],
           [ 5.2856e-02, -3.5675e-02,  3.4542e-03,  ...,  1.6983e-02,
             9.6436e-03, -4.9286e-03],
           [-1.4565e-02, -2.0508e-02,  9.6035e-04,  ...,  3.5324e-03,
            -4.6921e-03, -2.1000e-03]]], device='cuda:0', dtype=torch.float16),
  tensor([[[ 0.0025, -0.0373, -0.0152,  ..., -0.0137,  0.0076, -0.0085],
           [ 0.0701,  0.0117, -0.0166,  ..., -0.0080, -0.0960, -0.0045],
           [-0.0323, -0.0509, -0.0232,  ..., -0.0356,  0.0202,  0.0189],
           ...,
           [-0.0354, -0.0089,  0.0213,  ...,  0.0497, 

In [ ]:
hidden_states[0][1][0][0][4095]

tensor(-0.0085, device='cuda:0', dtype=torch.float16)

other copy

In [ ]:
input_prompt = "Pretend to be Sherlock Holmes, what is your expertise? Please answer in one word.2"
data = torch.load("/content/drive/MyDrive/llm/activations/{}.pt".format(input_prompt.replace(' ', '_'), map_location=torch.device('cpu')))

prompt = data['prompt']
hidden_states = data['hidden_states']
output_sequence = data['sequences'][0]
output = data['output'].split("Response:")[1]

In [ ]:
hidden_states

((tensor([[[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
           [ 0.0250,  0.0056, -0.0069,  ...,  0.0036, -0.0244,  0.0168],
           [-0.0126, -0.0181, -0.0089,  ..., -0.0017, -0.0070,  0.0112],
           ...,
           [-0.0136, -0.0088,  0.0062,  ...,  0.0504, -0.0424, -0.0055],
           [ 0.0529, -0.0357,  0.0035,  ...,  0.0170,  0.0096, -0.0049],
           [-0.0146, -0.0205,  0.0010,  ...,  0.0035, -0.0047, -0.0021]]],
         device='cuda:0', dtype=torch.float16),
  tensor([[[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
           [ 0.0701,  0.0117, -0.0166,  ..., -0.0080, -0.0960, -0.0045],
           [-0.0323, -0.0509, -0.0232,  ..., -0.0356,  0.0202,  0.0189],
           ...,
           [-0.0354, -0.0089,  0.0213,  ...,  0.0497, -0.0457, -0.0046],
           [ 0.0732, -0.0464,  0.0019,  ...,  0.0251,  0.0245, -0.0690],
           [-0.0273, -0.0312, -0.0263,  ..., -0.0122, -0.0017, -0.0118]]],
         device='cuda:0', dtype=torch.fl

In [ ]:
prompt

'Pretend to be Sherlock Holmes, what is your expertise? Please answer in one word.'

In [ ]:
output_sequence


tensor([    0, 13866,   338,   385, 15278,   393, 16612,   263,  3414, 29889,
        14350,   263,  2933,   393,  7128,  2486,  1614,  2167,   278,  2009,
        29889,    13,    13,  2277, 29937,  2799,  4080, 29901,    13, 29925,
         2267,   355,   304,   367, 17375,   908,  4168,  4467, 29892,   825,
          338,   596, 17924,   895, 29973,  3529,  1234,   297,   697,  1734,
        29889,    13,    13,  2277, 29937, 13291, 29901,    13,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0, 

In [ ]:
output

'\n ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇ '